In [ ]:
from init_notebook import *
from src.train.experiment import load_experiment_trainer
from experiments.denoise.resconv import *
from functools import partial

In [ ]:
def dump_module_stacktrace(model: nn.Module, *input):
    stack = []
    def _hook(model, args, kwargs=None, name: str=""):
        stack.append({
            "module": name,
            "input": ", ".join([
                str(arg.shape) if isinstance(arg, torch.Tensor) else type(arg).__name__
                for arg in args
            ]),
            "params": model.extra_repr(),
            
        })
        
    hooks = []
    def _register_hooks(model, path: List[str], idx: int):
        hooks.append(
            model.register_forward_pre_hook(partial(_hook, name=".".join(path)))
        )
        for name, child in model.named_children():
            _register_hooks(child, path + [name], idx + 1)

    _register_hooks(model, [type(model).__name__], 0)
    with torch.no_grad():
        model(*input)
    
    for hook in hooks:
        hook.remove()

    if stack:
        df = pd.DataFrame(stack)
        print(df.to_markdown(index=False))
    #for s in stack:
    #    print("{name:40} {input}".format(**s))


model = ResConv(3, num_layers=3, stride=2)
_ = dump_module_stacktrace(model, torch.randn(1, 3, 32, 32))